Scratch code for generalization to n_variant a/b test + analysis

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import matplotlib.ticker as mtick

from matplotlib import style
import statsmodels.api as sm
from statsmodels.stats.multitest import multipletests as mult_test
from tqdm.auto import tqdm

In [2]:
# additional imports 
sys.path.append('../../ab_testing_utils/')

In [3]:
import conversion_rate_utils

In [4]:
# Set pandas preferences
pd.options.display.max_columns=500
pd.options.display.max_colwidth=500
pd.options.display.max_rows=500

In [5]:
# Set plot style
style.use('fivethirtyeight')

In [6]:
# Initialize tqdm for pandas
tqdm.pandas()

We'll continue to work in units of weeks for the experiment run times.  Suppose that we already know the average baseline conversion rate, as well as the average number of daily visitors from historical data. There could be variance in these estimates, but we'll ignore that problem here for now

In [8]:
daily_num_observations = 10000
monthly_num_observations = daily_num_observations * 7 * 4
baseline_conversion_rate = 0.05

In [7]:
number_of_weeks_for_experiment = 4
number_of_days_for_experiment = number_of_weeks_for_experiment * 7

In [9]:
# First, we need to simulate the number of expected visitors per day
daily_units = []
day_list = []
for i in range(number_of_days_for_experiment):
    day_ = str(i)
    number_of_observations = np.random.poisson(daily_num_observations)
    observations = [1] * number_of_observations
    daily_units = daily_units + observations
    day_index = [day_] * number_of_observations
    day_list = day_list + day_index

In [10]:
df_ab = pd.DataFrame()
df_ab['day'] = day_list
df_ab['units'] = daily_units

In [11]:
df_ab.head()

,day,units
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1
